# 24. None과 독스트링을 사용해 동적인 디폴트 인자를 지정하라

종종 키워드 인자의 값으로 정적으로 정해지지 않는 타입의 값을 써야 할 때가 있다.

In [1]:
from time import sleep
from datetime import datetime

In [2]:
def log(message, when=datetime.now()):
    print(f'{when}: {message}')

In [3]:
log('안녕')

2021-02-23 00:57:48.924599: 안녕


In [4]:
log('다시 안녕!')

2021-02-23 00:57:48.924599: 다시 안녕!


디폴트 인자는 함수가 정의되는 시점에 한번 호출되므로 타임스탬프는 고정

디폴트 값으로 None을 지정하고 실제 동작을 독스트링에 문서화해야 함

In [5]:
def log(message, when=None):
    """메시지와 타임스탬프를 로그에 남긴다.
    
    Args:
        message: 출력할 메시지.
        when : 메시지가 발생한 시각(datetime).
            디폴트 값은 현재 시간이다.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

In [6]:
log('안녕')

2021-02-23 01:00:32.705004: 안녕


In [7]:
log('다시 안녕!')

2021-02-23 01:00:35.008693: 다시 안녕!


디폴트 인자 값으로 None을 사용하는 것은 인자가 가변적인 경우 특히 중요하다.

예를 들어 JSON 데이터로 인코딩된 값을 읽으려고 하는데, 데이터 디코딩에 실패하면 디폴트로 빈 딕셔너리를 반환하고 싶다.

In [8]:
import json

In [9]:
def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

In [10]:
foo = decode('잘못된 데이터')

In [11]:
foo['stuff'] = 5

In [12]:
bar = decode('또 잘못된 데이터')

In [13]:
bar['meep'] = 1

In [14]:
print(foo)

{'stuff': 5, 'meep': 1}


In [15]:
print(bar)

{'stuff': 5, 'meep': 1}


딕셔너리가 디폴트 파라미터로 같기 때문에 동일한 객체를 쓰게된다.

In [16]:
def decode(data, default=None):
    """문자열로부터 JSON 데이터를 읽어온다.
        
    Args:
        data: 디코딩할 JSON 데이터.
        default: 디코딩 실패 시 반환할 값이다.
            디폴트 값은 빈 딕셔너리다.
    """
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

In [17]:
foo = decode('잘못된 데이터')
foo['stuff'] = 5
bar = decode('또 잘못된 데이터')
bar['meep'] = 1

In [18]:
print(foo)
print(bar)

{'stuff': 5}
{'meep': 1}


이 방법은 타입 애너테이션을 사용해도 잘 작동한다.

In [19]:
from typing import Optional

In [20]:
def log_typed(message: str, when: Optional[datetime]=None) -> None:
    """메시지와 타임스탬프를 로그에 남긴다.
    
    Args:
        message: 출력할 메시지.
        when : 메시지가 발생한 시각(datetime).
            디폴트 값은 현재 시간이다.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

In [21]:
log_typed('안녕')

2021-02-23 01:06:29.832369: 안녕


## 기억해야 할 내용
- 디폴트 인자 값은 그 인자가 포함된 함수 정의가 속한 모듈이 로드되는 시점에 단 한 번만 평가 된다. 이로 인해 동적인 값({}, [], datetime.now() 등)의 경우 이상한 동작이 일어날 수 있다.
- 동적인 값을 가질 수 있는 키워드 인자의 디폴트 값을 표현할 때는 None을 사용하라, 그리고 함수의 독스트링에 실제 동적인 디폴트 인자가 어떻게 동작하는지 문서화해두라.
- 타입 애너테이션을 사용할 때도 None을 사용해 키워드 인자의 디폴트 값을 표현하는 방식을 적용할 수 있다.